## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.append("C:/Users/raney/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/raney/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,63.81,74,85,14.88,overcast clouds
1,1,Algiers,DZ,36.7525,3.0420,57.02,94,75,6.91,moderate rain
2,2,Mont-De-Marsan,FR,44.0000,-0.6667,60.28,57,0,6.91,clear sky
3,3,Hithadhoo,MV,-0.6000,73.0833,83.44,68,0,13.27,clear sky
4,4,Daru,PG,-9.0763,143.2092,76.89,89,100,5.17,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             69
City                   69
Country                69
Lat                    69
Lng                    69
Max Temp               69
Humidity               69
Cloudiness             69
Wind Speed             69
Current Description    69
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Chuy,UY,71.28,overcast clouds,-33.6971,-53.4616,None
26,Mentakab,MY,73.87,clear sky,3.4865,102.3516,None
49,Kiama,AU,68.00,overcast clouds,-34.6833,150.8667,None
50,Taltal,CL,69.26,few clouds,-25.4000,-70.4833,None
58,Umm Lajj,SA,68.54,clear sky,25.0213,37.2685,None


In [8]:
#define a function to find the nearest search type

def findnearest(params):
    
     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

     # 6e. Make request and retrieve the JSON data from the search.         
    searchentities = requests.get(base_url, params=params).json()
    
    # 6f. Get the first entity from the results
    entity_name = searchentities["results"][0]["name"]
    entity_address = searchentities["results"][0]["vicinity"]    

    #Combine the name and address return
    name_address = f"{entity_name}~{entity_address}"
    
    return name_address

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
        
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    try:
        
        #Call the function to get the hotel name, address, and rating
        hotel_name_address = findnearest(params)
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        
        hotel_df.loc[index,"Hotel Name"] = hotel_name_address.split("~")[0]
    
    except (IndexError):
        
        print(f"Hotel not found for {city}... skipping.")


Hotel not found for Beloha... skipping.
Hotel not found for Kampene... skipping.
Hotel not found for Elko... skipping.
Hotel not found for Los Zacatones... skipping.
Hotel not found for Wagar... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/raney/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))